# KdV Solitons
An important equation from the theory of solitons is the Korteweg-deVries (KdV) equation,
$$ \frac{\partial \rho}{\partial t} = -6\rho\frac{\partial \rho}{\partial x} - \frac{\partial^3 \rho}{\partial^3 x} $$
Write a program that solves the KdV equation using the explicit/implicit scheme
$$
\frac{\rho^{n+1}_j-\rho^{n}_j}{\Delta t} = -6D_j\,\rho^n_j-\frac{1}{2}

\Biggl(
\frac{\rho^{n}_{j+2}-2\rho^{n}_{j+1}+2\rho^{n}_{j-1}-\rho^{n}_{j-2}}{2\Delta x^3} + \frac{\rho^{n+1}_{j+2}-2\rho^{n+1}_{j+1}+2\rho^{n+1}_{j-1}-\rho^{n+1}_{j-2}}{2\Delta x^3}
\Biggr)
$$

where
$$
D_j = \frac{\rho^{n}_{j+1}-\rho^{n}_{j-1}}{2\Delta x}
$$

Use the Dirichlet boundary conditions $\rho(x = ±L/2) = 0$. Use the solitary wave solution of the KdV equation as an initial condition, $\rho(x, t) = 2\mbox{sech}^2(x − 4t)$. What do you observe?


In [1]:
import numpy as np

#parameters
tau=0.01
maxTime=10
tGridNum=maxTime//tau #the // automatically floors the result, so we get an integer length

L=1 #length
xGridSize=0.01
xGridNum=L//xGridSize # integer

#create grid

grid=np.zeros((xGridNum,tGridNum))



NameError: name 'XGridSize' is not defined